In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [2]:
import tensorflow as tf
import os

# Specify the classes to include (excluding the specified fruits/vegetables)
included_classes = [
    'freshbanana',
    'freshcapsicum',
    'freshcucumber',
    'freshoranges',
    'freshpotato',
    'freshtomato',
    'rottenbanana',
    'rottencapsicum',
    'rottencucumber',
    'rottenoranges',
    'rottenpotato',
    'rottentomato'
]

img_height, img_width = 180, 180
batch_size = 64
dataset_dir = "/kaggle/input/fresh-and-stale-classification/dataset/Train"

# Function to count the number of images in each class
def count_images_per_class(directory, classes):
    class_counts = {}
    for class_name in classes:
        class_dir = os.path.join(directory, class_name)
        if os.path.exists(class_dir):
            image_count = len(os.listdir(class_dir))
            class_counts[class_name] = image_count
        else:
            class_counts[class_name] = 0  # If the class directory doesn't exist
    return class_counts

# Count images in each class
image_counts = count_images_per_class(dataset_dir, included_classes)

# Print the counts for each class
print("Number of images per class:")
for class_name, count in image_counts.items():
    print(f"{class_name}: {count} images")

# Load the training dataset while excluding certain classes
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    class_names=included_classes  # Include only the specified classes
)

# Load the validation dataset using the same classes
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    class_names=included_classes  # Include only the specified classes
)


Number of images per class:
freshbanana: 2468 images
freshcapsicum: 990 images
freshcucumber: 496 images
freshoranges: 1466 images
freshpotato: 536 images
freshtomato: 1858 images
rottenbanana: 2932 images
rottencapsicum: 901 images
rottencucumber: 421 images
rottenoranges: 1595 images
rottenpotato: 802 images
rottentomato: 1825 images
Found 16290 files belonging to 12 classes.
Using 13032 files for training.
Found 16290 files belonging to 12 classes.
Using 3258 files for validation.


In [3]:
from tensorflow.keras import layers, models

# Load the pretrained ResNet50 model without the top layer
pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(180, 180, 3),
    pooling='avg',
    weights='imagenet'
)

# Freeze the layers in the pretrained model
for layer in pretrained_model.layers:
    layer.trainable = False

# Create a new input layer
inputs = layers.Input(shape=(180, 180, 3))

# Pass the inputs through the pretrained model
x = pretrained_model(inputs)

# Add your custom layers on top
x = layers.Dense(512, activation='relu')(x)
outputs = layers.Dense(12, activation='softmax')(x)

# Create the full model
resnet_model = models.Model(inputs, outputs)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
resnet_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 2048)           │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 12)             │         6,156 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,642,956 (94.01 MB)

 Trainable params: 1,055,244 (4.03 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [5]:
from tensorflow.keras.callbacks import EarlyStopping
# Compile the model
resnet_model.compile(
    optimizer=Adam(learning_rate=0.001),  # Optimizer with the initial learning rate
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [6]:
epochs=10
history = resnet_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
    callbacks=EarlyStopping(
    monitor='val_loss',  # You can also monitor 'val_accuracy'
    patience=3,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restores the model weights from the best epoch
)
)

Epoch 1/10


I0000 00:00:1729427630.005191      81 service.cc:145] XLA service 0x7f6538012140 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729427630.005255      81 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  3/204 ━━━━━━━━━━━━━━━━━━━━ 12s 64ms/step - accuracy: 0.1641 - loss: 3.0247

I0000 00:00:1729427635.261766      81 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


204/204 ━━━━━━━━━━━━━━━━━━━━ 59s 228ms/step - accuracy: 0.8371 - loss: 0.6041 - val_accuracy: 0.9678 - val_loss: 0.0923
Epoch 2/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 22s 108ms/step - accuracy: 0.9759 - loss: 0.0703 - val_accuracy: 0.9794 - val_loss: 0.0654
Epoch 3/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 22s 107ms/step - accuracy: 0.9865 - loss: 0.0417 - val_accuracy: 0.9850 - val_loss: 0.0441
Epoch 4/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 22s 106ms/step - accuracy: 0.9883 - loss: 0.0330 - val_accuracy: 0.9843 - val_loss: 0.0437
Epoch 5/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 28s 138ms/step - accuracy: 0.9903 - loss: 0.0295 - val_accuracy: 0.9847 - val_loss: 0.0449
Epoch 6/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 23s 110ms/step - accuracy: 0.9946 - loss: 0.0172 - val_accuracy: 0.9868 - val_loss: 0.0418
Epoch 7/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 22s 108ms/step - accuracy: 0.9952 - loss: 0.0137 - val_accuracy: 0.9800 - val_loss: 0.0549
Epoch 8/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 21s 105ms/step - accuracy: 0.9926 - loss: 0.0226 - val

In [7]:
# Save model weights with the required filename ending
model_weights_path = '/kaggle/working/resnet_model_weights.weights.h5'  # Ensure it ends with .weights.h5
resnet_model.save_weights(model_weights_path)
print(f"Model weights saved to {model_weights_path}")


Model weights saved to /kaggle/working/resnet_model_weights.weights.h5


In [8]:
import shutil

# Zip the model weights
shutil.make_archive('/kaggle/working/resnet_model_weights', 'zip', '/kaggle/working', 'resnet_model_weights.weights.h5')
print("Model weights zipped and ready for download.")


Model weights zipped and ready for download.


In [9]:
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import class_weight

# Number of images per class
class_counts = {
    'freshbanana': 2468,
    'freshcapsicum': 990,
    'freshcucumber': 496,
    'freshoranges': 1466,
    'freshpotato': 536,
    'freshtomato': 1858,
    'rottenbanana': 2932,
    'rottencapsicum': 901,
    'rottencucumber': 421,
    'rottenoranges': 1595,
    'rottenpotato': 802,
    'rottentomato': 1825
}

# Total number of images
total_images = sum(class_counts.values())
# Calculate class weights
class_weights = {i: total_images / (len(class_counts) * count) for i, count in enumerate(class_counts.values())}

# Load the pretrained ResNet50 model without the top layer
pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(180, 180, 3),
    pooling='avg',
    weights='imagenet'
)

# Freeze the layers in the pretrained model
for layer in pretrained_model.layers:
    layer.trainable = False

# Create a new input layer
inputs = layers.Input(shape=(180, 180, 3))

# Pass the inputs through the pretrained model
x = pretrained_model(inputs)

# Add your custom layers on top
x = layers.Dense(512, activation='relu')(x)
outputs = layers.Dense(len(class_counts), activation='softmax')(x)  # Adjusted for number of classes

# Create the full model
resnet_model = models.Model(inputs, outputs)

# Compile the model
resnet_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Define the number of epochs
epochs = 10

# Train the model with class weights
history = resnet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    class_weight=class_weights,  # Add class weights here
    callbacks=[EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )]
)

# Print the model summary
resnet_model.summary()


Epoch 1/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 37s 134ms/step - accuracy: 0.8415 - loss: 0.6390 - val_accuracy: 0.9699 - val_loss: 0.0937
Epoch 2/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 22s 106ms/step - accuracy: 0.9783 - loss: 0.1066 - val_accuracy: 0.9776 - val_loss: 0.0679
Epoch 3/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 41s 107ms/step - accuracy: 0.9840 - loss: 0.0841 - val_accuracy: 0.9807 - val_loss: 0.0576
Epoch 4/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 21s 105ms/step - accuracy: 0.9883 - loss: 0.0560 - val_accuracy: 0.9724 - val_loss: 0.0986
Epoch 5/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 22s 107ms/step - accuracy: 0.9903 - loss: 0.0405 - val_accuracy: 0.9828 - val_loss: 0.0540
Epoch 6/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 22s 108ms/step - accuracy: 0.9940 - loss: 0.0307 - val_accuracy: 0.9862 - val_loss: 0.0359
Epoch 7/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 22s 107ms/step - accuracy: 0.9923 - loss: 0.0274 - val_accuracy: 0.9776 - val_loss: 0.0676
Epoch 8/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 22s 109ms/step - accuracy: 0.9958 - loss: 0

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 2048)           │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 12)             │         6,156 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,753,446 (102.06 MB)

 Trainable params: 1,055,244 (4.03 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

 Optimizer params: 2,110,490 (8.05 MB)

In [10]:
# Save model weights with the required filename ending
model_weights_path = '/kaggle/working/resnet_model_weights.weights.h5'  # Ensure it ends with .weights.h5
resnet_model.save_weights(model_weights_path)
print(f"Model weights saved to {model_weights_path}")


Model weights saved to /kaggle/working/resnet_model_weights.weights.h5
